## Introducción al Machine Learning
### Características para Series de Precios

En este cuaderno se presentan algunos ejemplos de construccion de características
a partir de series de precios 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Datos del IBEX 35

In [ ]:
import pickle

with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
tef = stock_data['TEF']

In [ ]:
tef.head()

#### Características de fechas

Los días del año y los meses son propiedades que se repiten con los ciclos del año. Si queremos utilizarlas 
como características podemos representarlas de modo que respeten la relación de distancia entre ellos.  
Lo resolvemos construyendo 2 características que representan la posición en una circunferencia. De esta forma el mes 1 y el mes 12 tienen una distancia menor que de 1 a 3.  

In [ ]:
mes = tef.index.month
tef['mes_x1'] = np.sin(mes * (2* np.pi/12))
tef['mes_x2'] = np.cos(mes * (2* np.pi/12))

In [ ]:
tef

In [ ]:
tef_subset = tef.loc["2015-01-01":]
tef_subset[['mes_x1', 'mes_x2']].plot()


In [ ]:
def euclidean_distance(serie_a, serie_b):
    cuadrado_diff = (serie_a - serie_b)**2
    return np.sqrt(cuadrado_diff.sum())

In [ ]:
mes1 = tef.loc['2015-01-02',['mes_x1','mes_x2']]
mes3 = tef.loc['2015-03-02',['mes_x1','mes_x2']]
mes12 = tef.loc['2015-12-02',['mes_x1','mes_x2']]

In [ ]:
print(euclidean_distance(mes1, mes3))
print(euclidean_distance(mes1, mes12))

___

#### Características basadas en eventos

In [ ]:
tef = stock_data['TEF'].close

Primero calculamos dos medias móviles, una rápida de 10
sesiones y otra lenta de 50 sesiones.  Nos interesa identificar
los días de curce

In [ ]:
sma10 = tef.rolling(10).mean()
sma50 = tef.rolling(50).mean()

primero vemos los días que la rápida está por encima.
Representaremos con 0 o 1

In [ ]:
sma_above = (sma10 > sma50).astype(np.float)
sma_above.iloc[90:110]

Si hacemos la diferencia tendremos
- De 1 a 0: 1 la media rápida ha cruzado hacia arriba la larga
- De 0 a 1: -1 la media rápida ha cruzado hacia abajo la larga
- De 0 a 0, o de 1 a 1: 0 no hay cruce de medias    

In [ ]:
events_sma = sma_above.diff()
events_sma.iloc[90:110]

___

### Ejercicio Propuesto

Generar una característica que codifique el evento en el que la volatilidad de una serie supere o disminuya de un umral, por ejemplo 0.25 anualizado 